In [13]:
%pylab inline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import statsmodels
import statsmodels.api as sm
import scipy as sc
from sklearn import metrics
import itertools
import statsmodels.formula.api as smf
import sklearn.linear_model
import sklearn.preprocessing
from IPython.display import clear_output
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import ParameterGrid
from xgboost import XGBRegressor
import pickle

Populating the interactive namespace from numpy and matplotlib


c:\python27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Читаем подготовленные данные для сто двух ячеек регионов с каггл

In [2]:
data = pd.read_pickle('TaxiData/stats_kaggle_2015-03_2016-06.dat')
print data.shape
data.head()

(1194522, 2)


,region,count
tpep_pickup_datetime,,
2015-03-01,1075,47.0
2015-03-01,1076,103.0
2015-03-01,1077,51.0
2015-03-01,1125,99.0
2015-03-01,1126,287.0


Составим признаки для регрессии

In [3]:
def getDayPart(date_time):
   
    if (date_time.weekday() == 6 or date_time.weekday() == 5):
        if 4 <= date_time.hour and date_time.hour <= 8:
            return 0;
        if 9 <= date_time.hour and date_time.hour <= 18:
            return 1;
        if 19 <= date_time.hour or date_time.hour <= 5:
            return 0;
        
    else:
        if 1 <= date_time.hour and date_time.hour <= 6:
            return 0;
        if 7 <= date_time.hour and date_time.hour <= 18:
            return 1;
        if 19 <= date_time.hour and date_time.hour <= 23:
            return 2;
        if 23 < date_time.hour or date_time.hour == 0:
            return 0;


Составим признаки для регресии по каждому региону. Данные будем помещать в отдельный фрейм.

In [4]:
nModels = 6

K = 30

start_time = pd.to_datetime('2015-04-01 00:00:00')
end_time = pd.to_datetime('2016-06-30 17:00:00')

frames = {}

for reg in data.region.unique():
    #print reg
    sample_data = data[data.region == reg]['count'].sort_index()
    
    #fix 2016-06-30 23:00:00
    sample_data[pd.to_datetime('2016-06-30 23:00:00')] = sample_data[pd.to_datetime('2016-06-30 22:00:00')]
    
    frame = pd.DataFrame()

    frame['data'] = sample_data[start_time:end_time]

    for k in range(1, K + 1):
        frame.loc[:,'s'+ str(k)] = np.sin(np.arange(frame.shape[0]) * 2 * np.pi * k / 168)
        frame.loc[:,'c'+ str(k)] = np.cos(np.arange(frame.shape[0]) * 2 * np.pi * k / 168)

    frame.loc[:, 'day_part'] = frame.index.map(getDayPart)
    frame.loc[:, 'day'] = frame.index.map(lambda x: x.day)
    frame.loc[:, 'month'] = frame.index.map(lambda x: x.month)
    frame.loc[:, 'weekday'] = frame.index.map(lambda x: x.weekday())

    Khist = 48

    for k in range(0, Khist + 1):
        t_delta = datetime.timedelta(hours=k)
        frame.loc[:, 'data_history_' + str(k)] =  sample_data[start_time - t_delta:end_time - t_delta].values

    for shift in range(1, nModels + 1):
        delta = datetime.timedelta(hours=shift)
        frame['target_' + str(shift)] = sample_data[start_time + delta:end_time + delta].values
        
    clear_output(wait=True)
    print 'processing region ', reg
    frames[reg] = frame

processing region  1684


Обучим модели на данных до конца апреля включительно (последнее Т это '2016-04-30 17:00:00' соответственно последний отклик для T+6 это отклик на '2016-04-30 23:00:00'. Используем регресиию Лассо с регуляризацией L1. В цикле обучение будем подбирать параметр альфа в диапазоне от 0 до 1 (диапазон выбран эмпирически на отдельных рядах ). 
Для теста используем уже май.

In [6]:
%%time

train_from = pd.to_datetime('2015-04-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

rates = np.linspace(0.1, 0.25, 25)
topMaes = []
topRegionRates = {}

for reg in data.region.unique():
    
    regionRates = {}  
    regMaes = []
    
    print 'region', reg,
    
    for shift in range(1, nModels + 1):
        
        bestRate = 0
        bestMae = sys.maxint
        
        for rate in rates:
            
            m = XGBRegressor(#n_jobs=8,
                            tree_method='gpu_hist', learning_rate=rate)

            trainF = frames[reg][train_from : train_to]
            fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

            mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                              fitted.predict(frames[reg][test_from : test_to][x_names]))
            if mae < bestMae:
                bestMae = mae
                bestRate = rate
        
        regionRates[shift] = bestRate
        regMaes.append(bestMae)
        
    print 'mean mae', np.mean(regMaes), 'rates ', regionRates.values()
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionRates[reg] = regionRates

region 1075 mean mae 10.453595522318976 rates  [0.23124999999999998, 0.19374999999999998, 0.24375, 0.1375, 0.21875, 0.2125]
region 1076 mean mae 15.995231138702765 rates  [0.16875, 0.16875, 0.16875, 0.19374999999999998, 0.22499999999999998, 0.23124999999999998]
region 1077 mean mae 11.507348878706528 rates  [0.2125, 0.19374999999999998, 0.18125, 0.13125, 0.18125, 0.20625]
region 1125 mean mae 9.961423173728267 rates  [0.21875, 0.14375, 0.23124999999999998, 0.1875, 0.20625, 0.2125]
region 1126 mean mae 20.620928253012302 rates  [0.18125, 0.25, 0.2125, 0.19374999999999998, 0.23124999999999998, 0.1625]
region 1127 mean mae 25.86699343996646 rates  [0.2125, 0.1875, 0.20625, 0.20625, 0.2375, 0.2375]
region 1128 mean mae 30.411758695575745 rates  [0.2, 0.25, 0.19374999999999998, 0.21875, 0.1875, 0.19374999999999998]
region 1129 mean mae 33.42530034348726 rates  [0.20625, 0.24375, 0.24375, 0.2375, 0.24375, 0.22499999999999998]
region 1130 mean mae 42.304906226946635 rates  [0.14375, 0.21875, 

region 1426 mean mae 1.9413066537303865 rates  [0.2125, 0.1875, 0.14375, 0.20625, 0.13125, 0.1875]
region 1431 mean mae 4.698309114294434 rates  [0.175, 0.2125, 0.1125, 0.10625000000000001, 0.1, 0.10625000000000001]
region 1434 mean mae 2.848793415689608 rates  [0.14375, 0.1875, 0.16875, 0.15625, 0.1875, 0.15]
region 1435 mean mae 2.5770919564934966 rates  [0.14375, 0.125, 0.1, 0.2, 0.18125, 0.1125]
region 1436 mean mae 5.453711556269879 rates  [0.15625, 0.2, 0.19374999999999998, 0.15625, 0.15, 0.15]
region 1437 mean mae 6.632463869771068 rates  [0.1875, 0.15, 0.16875, 0.1375, 0.2375, 0.20625]
region 1438 mean mae 2.946531119301451 rates  [0.20625, 0.20625, 0.2125, 0.175, 0.20625, 0.2]
region 1439 mean mae 2.484341953727755 rates  [0.1875, 0.11875000000000001, 0.23124999999999998, 0.13125, 0.18125, 0.15]
region 1441 mean mae 2.086534034845578 rates  [0.1375, 0.1625, 0.15625, 0.1625, 0.20625, 0.21875]
region 1442 mean mae 2.6704865159163567 rates  [0.22499999999999998, 0.2375, 0.24375, 

In [5]:
#with open('TaxiData/topRegionRates.dat', 'wb') as f:
#    pickle.dump(topRegionRates, f)

with open('TaxiData/topRegionRates.dat', 'r') as f:
    topRegionRates = pickle.load(f)
    
#np.mean(map(lambda x: x[1], topMaes))

In [10]:
%%time

train_from = pd.to_datetime('2015-04-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

estimators = np.append(np.arange(10, 100, 10), np.arange(100, 550, 50))

topMaes = []
topRegionEstimators = {}

for reg in data.region.unique():
    
    regionEsts = {}  
    regMaes = []
    
    print 'region', reg,
    
    for shift in range(1, nModels + 1):
        
        bestEst = 0
        bestMae = sys.maxint
        
        for est in estimators:
            
            m = XGBRegressor(#n_jobs=8, 
                            tree_method='gpu_hist',
                            learning_rate=topRegionRates[reg][shift], n_estimators=est)

            trainF = frames[reg][train_from : train_to]
            fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

            mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                              fitted.predict(frames[reg][test_from : test_to][x_names]))
            
            if mae < bestMae:
                bestMae = mae
                bestEst = est
        
        regionEsts[shift] = bestEst
        
        regMaes.append(bestMae)
        
    print 'mean mae', np.mean(regMaes), 'estimators ', regionEsts.values()
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionEstimators[reg] = regionEsts

 region 1075 mean mae 10.361408573257316 estimators  [100, 250, 150, 350, 100, 250]
region 1076 mean mae 15.867448276329748 estimators  [350, 150, 150, 300, 200, 100]
region 1077 mean mae 11.355386593862596 estimators  [90, 300, 250, 350, 300, 300]
region 1125 mean mae 9.779276998921134 estimators  [400, 500, 300, 250, 350, 200]
region 1126 mean mae 20.245658136349732 estimators  [350, 450, 300, 250, 150, 250]
region 1127 mean mae 25.164229893252877 estimators  [500, 300, 350, 350, 300, 500]
region 1128 mean mae 29.4228107678659 estimators  [350, 400, 350, 300, 500, 500]
region 1129 mean mae 32.16800309087868 estimators  [250, 350, 450, 400, 500, 300]
region 1130 mean mae 40.90542839387541 estimators  [500, 200, 500, 350, 500, 500]
region 1131 mean mae 20.404106574454822 estimators  [250, 500, 350, 500, 500, 450]
region 1132 mean mae 10.981166837435344 estimators  [200, 300, 300, 450, 200, 100]
region 1172 mean mae 2.695051054663899 estimators  [90, 80, 90, 100, 80, 60]
region 1173 mea

In [7]:
#with open('TaxiData/topRegionEstimators.dat', 'wb') as f:
#    pickle.dump(topRegionEstimators, f)

with open('TaxiData/topRegionEstimators.dat', 'r') as f:
    topRegionEstimators = pickle.load(f)

#np.mean(map(lambda x: x[1], topMaes))

In [45]:
#topRegionEstimators


array([0.4       , 0.43529412, 0.47058824, 0.50588235, 0.54117647,
       0.57647059, 0.61176471, 0.64705882, 0.68235294, 0.71764706,
       0.75294118, 0.78823529, 0.82352941, 0.85882353, 0.89411765,
       0.92941176, 0.96470588, 1.        ])

In [46]:
%%time

#GRID SEARCH

train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

params = {
        'gamma': np.arange(0, 101, 25),
        'subsample': np.linspace(0.4, 1, 8),
        'colsample_bytree':  np.linspace(0.5, 1, 8),
        'max_depth': np.arange(1, 11, 1)
        }

topMaes = []
topRegionGrid = {}
gridParams = ParameterGrid(params)
print 'total params: ', len(gridParams)
for reg in data.region.unique():
    
    print 'region', reg,
    
    regMaes = []
    topGrid = {}
    trainF = frames[reg][train_from : train_to]
    
    for shift in range(1, nModels + 1):
        
        print 'shift', shift
        
        m = XGBRegressor(#n_jobs=8, 
                        tree_method='gpu_hist', learning_rate=topRegionRates[reg][shift],
                         n_estimators=topRegionEstimators[reg][shift])
        
        bestMae = sys.maxint
        bestGrid = None
        cnt = 0
        for g in gridParams:
            print '{0}\r'.format(cnt),
            cnt = cnt + 1
            
            m.set_params(**g)
            fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])
            mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                          fitted.predict(frames[reg][test_from : test_to][x_names]))
            if mae < bestMae:
                bestMae = mae
                bestGrid = g
        
        topGrid[shift] = bestGrid
        regMaes.append(bestMae)
        print 'mae', bestMae
                
    print 'mean mae', np.mean(regMaes), topGrid
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionGrid[reg] = topGrid

total params:  3200
region 1075 shift 1
mae 9.573107373649922
shift 2
mae 10.477991232530028
shift 3
mae 10.92397003708576
shift 4
mae 10.866701468563209
shift 5
mae 11.047376314503254
shift 6


KeyboardInterrupt: 

In [14]:
%%time

train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')


depths = np.arange(1, 11, 1)
topMaes = []
topRegionDepths = {}

for reg in data.region.unique():
    
    regionDepths = {}  
    regMaes = []
    
    print 'region', reg,
    
    for shift in range(1, nModels + 1):
        
        bestDepth = 0
        bestMae = sys.maxint
        
        for depth in depths:
            
            m = XGBRegressor(#n_jobs=8, 
                            tree_method='gpu_hist', learning_rate=topRegionRates[reg][shift],
                             n_estimators=topRegionEstimators[reg][shift], max_depth = depth)

            trainF = frames[reg][train_from : train_to]
            fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

            mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                              fitted.predict(frames[reg][test_from : test_to][x_names]))
            
            if mae < bestMae:
                bestMae = mae
                bestDepth = depth
        
        regionDepths[shift] = bestDepth
        
        regMaes.append(bestMae)
        
    print 'mean mae', np.mean(regMaes), 'n_estimators ', regionDepths.values()
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionDepths[reg] = regionDepths

 region 1075 mean mae 10.94448235818514 rates  [3, 3, 2, 3, 3, 9]
region 1076 mean mae 16.81779741340249 rates  [4, 2, 5, 5, 6, 5]
region 1077 mean mae 11.958925699160666 rates  [4, 8, 6, 6, 6, 4]
region 1125 mean mae 10.453116209538175 rates  [5, 2, 2, 8, 6, 6]
region 1126 mean mae 20.827191097124988 rates  [9, 9, 2, 9, 7, 9]
region 1127 mean mae 29.18935925228186 rates  [5, 9, 8, 10, 10, 5]
region 1128 mean mae 34.43882088699909 rates  [2, 5, 4, 6, 6, 2]
region 1129 mean mae 36.86870195030037 rates  [6, 9, 6, 7, 6, 6]
region 1130 mean mae 47.44310099163408 rates  [4, 5, 3, 3, 7, 8]
region 1131 mean mae 23.352959329910146 rates  [5, 7, 3, 5, 4, 2]
region 1132 mean mae 12.268125245894566 rates  [3, 3, 3, 5, 4, 3]
region 1172 mean mae 2.8573058024620117 rates  [5, 4, 4, 4, 4, 4]
region 1173 mean mae 6.865027498354716 rates  [2, 3, 5, 3, 5, 4]
region 1174 mean mae 6.214453968503854 rates  [2, 6, 5, 2, 7, 2]
region 1175 mean mae 4.22380965127432 rates  [4, 6, 3, 8, 7, 4]
region 1176 mean 

KeyboardInterrupt: 

In [117]:
with open('TaxiData/topRegionDepths.dat', 'wb') as f:
    pickle.dump(topRegionDepths, f)
    
np.mean(map(lambda x: x[1], topMaes))


{1075: {1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 6},
 1076: {1: 6, 2: 3, 3: 5, 4: 4, 5: 5, 6: 5},
 1077: {1: 5, 2: 3, 3: 3, 4: 7, 5: 6, 6: 8},
 1125: {1: 5, 2: 6, 3: 4, 4: 3, 5: 5, 6: 7},
 1126: {1: 4, 2: 6, 3: 6, 4: 9, 5: 6, 6: 9},
 1127: {1: 4, 2: 6, 3: 6, 4: 9, 5: 7, 6: 8},
 1128: {1: 3, 2: 3, 3: 5, 4: 7, 5: 3, 6: 7},
 1129: {1: 5, 2: 3, 3: 3, 4: 3, 5: 5, 6: 6},
 1130: {1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3},
 1131: {1: 5, 2: 3, 3: 6, 4: 3, 5: 3, 6: 3},
 1132: {1: 3, 2: 3, 3: 3, 4: 5, 5: 4, 6: 5},
 1172: {1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3},
 1173: {1: 3, 2: 3, 3: 4, 4: 4, 5: 3, 6: 3},
 1174: {1: 3, 2: 8, 3: 6, 4: 3, 5: 5, 6: 3},
 1175: {1: 3, 2: 5, 3: 7, 4: 5, 5: 10, 6: 9},
 1176: {1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 5},
 1177: {1: 3, 2: 5, 3: 4, 4: 5, 5: 5, 6: 5},
 1178: {1: 3, 2: 7, 3: 6, 4: 5, 5: 3, 6: 3},
 1179: {1: 3, 2: 6, 3: 9, 4: 7, 5: 7, 6: 8},
 1180: {1: 2, 2: 3, 3: 3, 4: 3, 5: 3, 6: 6},
 1181: {1: 5, 2: 3, 3: 3, 4: 6, 5: 3, 6: 4},
 1182: {1: 3, 2: 4, 3: 3, 4: 3, 5: 6, 6: 3},
 1183: {1

In [119]:
%%time

train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

gammas = np.arange(0, 101, 10)
topMaes = []
topRegionGammas = {}

for reg in data.region.unique():
    
    regionGammas = {}  
    regMaes = []
    
    print 'region', reg,
    
    for shift in range(1, nModels + 1):
        
        bestGamma = 0
        bestMae = sys.maxint
        
        for gamma in gammas:
            
            m = XGBRegressor(n_jobs=8, learning_rate=topRegionRates[reg][shift],
                             n_estimators=topRegionEstimators[reg][shift],
                             max_depth = topRegionDepths[reg][shift], gamma = gamma)

            trainF = frames[reg][train_from : train_to]
            fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

            mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                              fitted.predict(frames[reg][test_from : test_to][x_names]))
            
            if mae < bestMae:
                bestMae = mae
                bestGamma = gamma
        
        regionGammas[shift] = bestGamma
        
        regMaes.append(bestMae)
        
    print 'mean mae', np.mean(regMaes), 'gammas ', regionGammas.values()
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionGammas[reg] = regionGammas

region 1075 mean mae 10.737607333639424 gammas  [10, 10, 0, 0, 0, 40]
region 1076 mean mae 16.381414317310423 gammas  [60, 10, 0, 0, 90, 20]
region 1077 mean mae 11.631444803264747 gammas  [90, 0, 0, 80, 20, 0]
region 1125 mean mae 10.278084495070821 gammas  [10, 80, 0, 0, 0, 30]
region 1126 mean mae 20.708233643180353 gammas  [40, 50, 0, 100, 10, 60]
region 1127 mean mae 28.744571018821222 gammas  [30, 30, 60, 90, 0, 20]
region 1128 mean mae 33.71249390465749 gammas  [0, 50, 0, 20, 10, 20]
region 1129 mean mae 36.011712972893065 gammas  [80, 0, 0, 0, 30, 20]
region 1130 mean mae 46.24961137061835 gammas  [90, 0, 90, 80, 20, 10]
region 1131 mean mae 22.587118129083365 gammas  [90, 100, 90, 90, 0, 0]
region 1132 mean mae 11.82137661133874 gammas  [50, 0, 0, 10, 0, 40]
region 1172 mean mae 2.873848341267605 gammas  [50, 10, 0, 0, 10, 0]
region 1173 mean mae 6.838307504321629 gammas  [0, 30, 60, 70, 80, 0]
region 1174 mean mae 6.032888386383431 gammas  [0, 10, 0, 0, 0, 0]
region 1175 mean

In [120]:
with open('TaxiData/topRegionGammas.dat', 'wb') as f:
    pickle.dump(topRegionGammas, f)
    
np.mean(map(lambda x: x[1], topMaes))

17.904242000383537

In [121]:
%%time

train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

subsamples = np.linspace(0.4, 1, 10)
topMaes = []
topRegionSubsamples = {}

for reg in data.region.unique():
    
    regionSubsamples = {}  
    regMaes = []
    
    print 'region', reg,
    
    for shift in range(1, nModels + 1):
        
        bestSubsample = 0
        bestMae = sys.maxint
        
        for subsample in subsamples:
            
            m = XGBRegressor(n_jobs=8, learning_rate=topRegionRates[reg][shift],
                             n_estimators=topRegionEstimators[reg][shift],
                             max_depth = topRegionDepths[reg][shift],
                             gamma = topRegionGammas[reg][shift],
                             subsample = subsample)

            trainF = frames[reg][train_from : train_to]
            fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

            mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                              fitted.predict(frames[reg][test_from : test_to][x_names]))
            
            if mae < bestMae:
                bestMae = mae
                bestSubsample = subsample
        
        regionSubsamples[shift] = bestSubsample
        
        regMaes.append(bestMae)
        
    print 'mean mae', np.mean(regMaes), 'subsamples ', regionSubsamples.values()
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionSubsamples[reg] = regionSubsamples

region 1075 mean mae 10.734729691613994 rates  [1.0, 1.0, 1.0, 1.0, 0.4666666666666667, 1.0]
region 1076 mean mae 16.34172927677282 rates  [1.0, 1.0, 1.0, 0.6000000000000001, 0.8666666666666667, 1.0]
region 1077 mean mae 11.616627287766555 rates  [1.0, 1.0, 1.0, 1.0, 1.0, 0.8]
region 1125 mean mae 10.264705000399038 rates  [1.0, 0.8666666666666667, 1.0, 1.0, 0.8666666666666667, 1.0]
region 1126 mean mae 20.555758409581852 rates  [1.0, 0.8, 1.0, 0.8, 0.8666666666666667, 0.9333333333333333]
region 1127 mean mae 28.470247030365723 rates  [0.8, 0.6666666666666667, 0.6666666666666667, 1.0, 0.8666666666666667, 0.6000000000000001]
region 1128 mean mae 33.5746335131551 rates  [1.0, 1.0, 0.5333333333333333, 1.0, 1.0, 1.0]
region 1129 mean mae 35.96923027417119 rates  [1.0, 0.8666666666666667, 1.0, 1.0, 1.0, 1.0]
region 1130 mean mae 46.24961137061835 rates  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
region 1131 mean mae 22.467493855810726 rates  [0.5333333333333333, 1.0, 0.8666666666666667, 1.0, 1.0, 1.0]


region 1480 mean mae 3.161187804277598 rates  [0.9333333333333333, 1.0, 0.7333333333333334, 0.4, 1.0, 0.6000000000000001]
region 1482 mean mae 3.0983886359562782 rates  [1.0, 1.0, 0.9333333333333333, 1.0, 1.0, 0.9333333333333333]
region 1483 mean mae 2.7957207816755876 rates  [1.0, 1.0, 1.0, 1.0, 1.0, 0.9333333333333333]
region 1530 mean mae 3.4652491663832574 rates  [1.0, 0.6000000000000001, 0.6000000000000001, 1.0, 1.0, 1.0]
region 1532 mean mae 2.3533767953642757 rates  [1.0, 1.0, 0.8666666666666667, 1.0, 1.0, 1.0]
region 1533 mean mae 3.098464046128978 rates  [1.0, 0.9333333333333333, 1.0, 1.0, 1.0, 1.0]
region 1580 mean mae 2.0059039381424695 rates  [1.0, 1.0, 0.4666666666666667, 0.8, 0.5333333333333333, 1.0]
region 1630 mean mae 1.9520146606416924 rates  [1.0, 1.0, 1.0, 0.8666666666666667, 0.6000000000000001, 1.0]
region 1733 mean mae 2.120706268256454 rates  [1.0, 0.6000000000000001, 0.4666666666666667, 1.0, 1.0, 1.0]
region 1734 mean mae 31.39182734549261 rates  [1.0, 0.8, 1.0,

In [122]:
with open('TaxiData/topRegionSubsamples.dat', 'wb') as f:
    pickle.dump(topRegionSubsamples, f)
    
np.mean(map(lambda x: x[1], topMaes))

17.848182399818892

In [123]:
%%time

train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

colsamplesByTree = np.linspace(0.5, 1, 20)
topMaes = []
topRegionColSamplesByTree = {}

for reg in data.region.unique():
    
    regionColSamplesByTree = {}  
    regMaes = []
    
    print 'region', reg,
    
    for shift in range(1, nModels + 1):
        
        bestColSamplesByTree = 0
        bestMae = sys.maxint
        
        for colsampleByTree in colsamplesByTree:
            
            m = XGBRegressor(n_jobs=8, learning_rate=topRegionRates[reg][shift],
                             n_estimators=topRegionEstimators[reg][shift],
                             max_depth = topRegionDepths[reg][shift],
                             gamma = topRegionGammas[reg][shift],
                             subsample = topRegionSubsamples[reg][shift],
                             colsample_bytree = colsampleByTree)

            trainF = frames[reg][train_from : train_to]
            fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

            mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                              fitted.predict(frames[reg][test_from : test_to][x_names]))
            
            if mae < bestMae:
                bestMae = mae
                bestColSamplesByTree = colsampleByTree
        
        regionColSamplesByTree[shift] = bestColSamplesByTree
        
        regMaes.append(bestMae)
        
    print 'mean mae', np.mean(regMaes), 'col samples ', regionColSamplesByTree.values()
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionColSamplesByTree[reg] = regionColSamplesByTree

region 1075 mean mae 10.684574574568167 rates  [0.6052631578947368, 0.5, 1.0, 0.8947368421052632, 1.0, 0.7105263157894737]
region 1076 mean mae 16.247692219405547 rates  [1.0, 1.0, 0.5, 1.0, 0.6052631578947368, 0.631578947368421]
region 1077 mean mae 11.57780458355036 rates  [1.0, 0.8947368421052632, 0.6578947368421053, 0.5526315789473684, 1.0, 0.7368421052631579]
region 1125 mean mae 10.219394377883512 rates  [0.7368421052631579, 1.0, 0.868421052631579, 0.8157894736842105, 1.0, 0.5263157894736842]
region 1126 mean mae 20.209697891798008 rates  [1.0, 0.5263157894736842, 1.0, 0.9473684210526315, 0.5526315789473684, 0.6578947368421053]
region 1127 mean mae 28.21919581929896 rates  [0.9210526315789473, 0.763157894736842, 0.8947368421052632, 0.5789473684210527, 0.9736842105263157, 0.9210526315789473]
region 1128 mean mae 33.19407039490917 rates  [0.8157894736842105, 1.0, 0.9473684210526315, 0.6052631578947368, 1.0, 0.7368421052631579]
region 1129 mean mae 35.67107788829262 rates  [1.0, 1.0

region 1383 mean mae 24.78775612986405 rates  [0.868421052631579, 1.0, 0.9473684210526315, 0.631578947368421, 0.868421052631579, 0.6578947368421053]
region 1384 mean mae 38.386817731188394 rates  [0.6578947368421053, 1.0, 1.0, 1.0, 0.7894736842105263, 1.0]
region 1385 mean mae 23.803387941623736 rates  [0.6578947368421053, 1.0, 1.0, 0.763157894736842, 0.5263157894736842, 0.5263157894736842]
region 1386 mean mae 6.720340474102483 rates  [0.5, 1.0, 1.0, 1.0, 0.6842105263157895, 1.0]
region 1387 mean mae 6.152375284056434 rates  [1.0, 0.5263157894736842, 1.0, 0.9210526315789473, 1.0, 0.6052631578947368]
region 1388 mean mae 5.213117228578112 rates  [0.5, 0.7105263157894737, 1.0, 1.0, 0.8421052631578947, 0.6052631578947368]
region 1389 mean mae 3.858237416452796 rates  [0.5526315789473684, 0.5526315789473684, 1.0, 1.0, 0.5, 1.0]
region 1390 mean mae 4.044317370876301 rates  [0.631578947368421, 0.5789473684210527, 1.0, 0.763157894736842, 1.0, 0.5789473684210527]
region 1426 mean mae 1.96892

In [124]:
with open('TaxiData/topRegionColSamplesByTree.dat', 'wb') as f:
    pickle.dump(topRegionColSamplesByTree, f)
    
np.mean(map(lambda x: x[1], topMaes))

17.72658498666901

Получим модели с подобранными параметрами

In [128]:
%%time

train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

topMaes = []
topRegionModels = {}

for reg in data.region.unique():
    
    regionModels = {}  
    regMaes = []
    
    print 'region', reg,
    
    for shift in range(1, nModels + 1):
        
        m = XGBRegressor(n_jobs=8, learning_rate=topRegionRates[reg][shift],
                         n_estimators=topRegionEstimators[reg][shift],
                         max_depth = topRegionDepths[reg][shift],
                         gamma = topRegionGammas[reg][shift],
                         subsample = topRegionSubsamples[reg][shift],
                         colsample_bytree = topRegionColSamplesByTree[reg][shift])

        trainF = frames[reg][train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                              fitted.predict(frames[reg][test_from : test_to][x_names]))
            
        regMaes.append(mae)
        regionModels[shift] = fitted
        
    print 'mean mae', np.mean(regMaes)
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionModels[reg] = regionModels

region 1075 mean mae 10.684574574568167
region 1076 mean mae 16.247692219405547
region 1077 mean mae 11.57780458355036
region 1125 mean mae 10.219394377883512
region 1126 mean mae 20.209697891798008
region 1127 mean mae 28.21919581929896
region 1128 mean mae 33.19407039490917
region 1129 mean mae 35.67107788829262
region 1130 mean mae 46.145980772156946
region 1131 mean mae 22.467493855810726
region 1132 mean mae 11.771088365564543
region 1172 mean mae 2.856059156556521
region 1173 mean mae 6.758510628690792
region 1174 mean mae 6.028089242861729
region 1175 mean mae 4.023086022425999
region 1176 mean mae 4.574880643207101
region 1177 mean mae 34.11372793392926
region 1178 mean mae 45.14503188836559
region 1179 mean mae 41.77691861773376
region 1180 mean mae 49.490480898305215
region 1181 mean mae 80.6119984414699
region 1182 mean mae 42.75353887589816
region 1183 mean mae 19.923391330978813
region 1184 mean mae 4.497835579363946
region 1221 mean mae 2.385935511172866
region 1222 mean 

In [130]:
with open('TaxiData/topRegionModels.dat', 'wb') as f:
    pickle.dump(topRegionModels, f)
    
np.mean(map(lambda x: x[1], topMaes))

17.72658498666901

Подсчитаем метрику за май

In [135]:
test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

alphas = np.arange(0, 1.01, 0.01)

totalMay = 0
regMaes = []
for reg in data.region.unique():
    
    regionModels = topRegionModels[reg]

    regMae = 0   
    
    for shift in range(1, nModels + 1):
        mae = metrics.mean_absolute_error(frames[reg][test_from:test_to]['target_' + str(shift)], 
                                          regionModels[shift].predict(frames[reg][test_from:test_to][x_names]))
        regMae += mae

    totalMay += regMae 
    regMaes.append(regMae)


In [142]:
totalMay / (102 * 6)

17.726584986669014

Обучим модели на данных до мая включительно для дальнейших предсказаний июня

In [140]:
%%time

train_from = pd.to_datetime('2016-01-01 00:00:00')
train_to = pd.to_datetime('2016-05-31 23:00:00')

x_names = []
for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

test_from = pd.to_datetime('2016-05-31 23:00:00')
test_to = pd.to_datetime('2016-06-30 17:00:00')

topMaes = []
topRegionModels = {}

for reg in data.region.unique():
    
    regionModels = {}  
    regMaes = []
    
    print 'region', reg,
    
    for shift in range(1, nModels + 1):
        
        m = XGBRegressor(n_jobs=8, learning_rate=topRegionRates[reg][shift],
                         n_estimators=topRegionEstimators[reg][shift],
                         max_depth = topRegionDepths[reg][shift],
                         gamma = topRegionGammas[reg][shift],
                         subsample = topRegionSubsamples[reg][shift],
                         colsample_bytree = topRegionColSamplesByTree[reg][shift])

        trainF = frames[reg][train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(frames[reg][test_from : test_to]['target_' + str(shift)],
                                              fitted.predict(frames[reg][test_from : test_to][x_names]))
            
        regMaes.append(mae)
        regionModels[shift] = fitted
        
    print 'mean mae', np.mean(regMaes)
    
    topMaes.append((regMaes, np.mean(regMaes)))
    topRegionModels[reg] = regionModels

 region 1075 mean mae 11.400537652930458
region 1076 mean mae 14.758978734816703
region 1077 mean mae 11.140214991507
region 1125 mean mae 9.064605056239175
region 1126 mean mae 19.927841422607848
region 1127 mean mae 25.792577186886803
region 1128 mean mae 30.104969132474526
region 1129 mean mae 34.90330232206758
region 1130 mean mae 43.526898693093614
region 1131 mean mae 20.207584859121653
region 1132 mean mae 11.882103327569707
region 1172 mean mae 2.177796518101003
region 1173 mean mae 5.348355150500655
region 1174 mean mae 6.2195768143469365
region 1175 mean mae 3.816429139253421
region 1176 mean mae 4.414221430436158
region 1177 mean mae 30.293458136756374
region 1178 mean mae 44.501030564697196
region 1179 mean mae 41.245394198544375
region 1180 mean mae 49.44094209859977
region 1181 mean mae 66.99783990855538
region 1182 mean mae 40.91476893213801
region 1183 mean mae 17.700507149329553
region 1184 mean mae 4.057104138852833
region 1221 mean mae 2.4785384149748686
region 1222 

In [141]:
with open('TaxiData/topRegionJuneModels.dat', 'wb') as f:
    pickle.dump(topRegionModels, f)
    
np.mean(map(lambda x: x[1], topMaes))

17.14897759458914

Подсчитаем метрику за июнь

In [154]:
test_from = pd.to_datetime('2016-05-31 23:00:00')
test_to = pd.to_datetime('2016-06-30 17:00:00')

alphas = np.arange(0, 1.01, 0.01)

totalJune = 0
regMaes = []
juneDf = pd.DataFrame()
juneDf['ix'] = frames[1075][test_from:test_to]['data']

for reg in data.region.unique():
    
    regionModels = topRegionModels[reg]

    regMae = 0   
    
    for shift in range(1, nModels + 1):
        preds = regionModels[shift].predict(frames[reg][test_from:test_to][x_names])
        juneDf[str(reg) + '_' + str(shift)] = preds
        mae = metrics.mean_absolute_error(frames[reg][test_from:test_to]['target_' + str(shift)], preds)
        regMae += mae

    totalJune += regMae 
    regMaes.append(regMae)


In [155]:
totalJune / (102 * 6)

17.14897759458915

In [1]:
juneDf.to_pickle('TaxiData/juneDf.dat')
juneDf.head()

NameError: name 'juneDf' is not defined

In [3]:
juneDf = pd.read_pickle('TaxiData/juneDf.dat')

In [7]:
juneDf = juneDf.sort_index()

In [9]:
juneDf.head()

,ix,1075_1,1075_2,1075_3,1075_4,1075_5,1075_6,1076_1,1076_2,1076_3,...,2168_3,2168_4,2168_5,2168_6,1684_1,1684_2,1684_3,1684_4,1684_5,1684_6
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,,,
2016-05-31 23:00:00,34.0,26.614845,12.607468,7.219707,4.224356,7.072586,8.135632,35.374245,26.591696,13.369905,...,0.517792,1.058056,5.594037,60.115234,0.560961,-0.303833,-0.464976,0.196178,0.087564,0.330099
2016-06-01 00:00:00,26.0,12.943633,5.808178,2.275108,6.100738,9.920300,26.070053,20.565361,10.574583,6.058395,...,0.463521,4.325445,64.223534,99.535583,1.153414,0.448262,2.077868,-0.329279,0.661815,3.809043
2016-06-01 01:00:00,14.0,5.864309,3.368098,3.219246,8.079777,19.629467,42.326790,8.070734,13.571685,11.238285,...,0.967022,63.951889,89.706772,83.803101,0.112471,0.333834,0.736918,-0.187371,6.295387,10.579295
2016-06-01 02:00:00,5.0,5.955711,7.060913,10.182145,20.162502,48.995415,76.717407,13.508312,15.419728,21.472664,...,64.457146,82.042053,73.418083,66.266167,0.338468,1.253895,0.635118,3.558313,9.790144,12.204899
2016-06-01 03:00:00,2.0,6.190418,10.945358,18.219942,39.151779,79.413147,68.422943,9.698759,25.049486,67.362640,...,81.973915,78.971977,72.077606,70.052742,0.156648,0.421206,6.293431,10.223243,12.744143,21.266201


Index([u'1075_1', u'1075_2', u'1075_3', u'1075_4', u'1075_5', u'1075_6',
       u'1076_1', u'1076_2', u'1076_3', u'1076_4',
       ...
       u'2168_3', u'2168_4', u'2168_5', u'2168_6', u'1684_1', u'1684_2',
       u'1684_3', u'1684_4', u'1684_5', u'1684_6'],
      dtype='object', length=612)

In [21]:
results = []
for index, row in juneDf.iterrows():
    for col in juneDf.columns:
        split = str.split(col,'_')
        results.append(( str.format("{0}_{1}_{2}_{3}",split[0], index.date(), index.hour, split[1]), row[col]))

In [22]:
juneFrame = pd.DataFrame(results)

In [24]:
juneFrame.to_csv('TaxiData/w6_v2_kaggle.csv',index=False)

In [47]:
juneDf = pd.read_pickle('TaxiData/juneDf.dat')

In [126]:
s = data[data.region == 1075]['2016-06-01 00:00:00':'2016-06-30 23:00:00']['count'].sort_index()
s.index

DatetimeIndex(['2016-06-01 00:00:00', '2016-06-01 01:00:00',
               '2016-06-01 02:00:00', '2016-06-01 03:00:00',
               '2016-06-01 04:00:00', '2016-06-01 05:00:00',
               '2016-06-01 06:00:00', '2016-06-01 07:00:00',
               '2016-06-01 08:00:00', '2016-06-01 09:00:00',
               ...
               '2016-06-30 13:00:00', '2016-06-30 14:00:00',
               '2016-06-30 15:00:00', '2016-06-30 16:00:00',
               '2016-06-30 17:00:00', '2016-06-30 18:00:00',
               '2016-06-30 19:00:00', '2016-06-30 20:00:00',
               '2016-06-30 21:00:00', '2016-06-30 22:00:00'],
              dtype='datetime64[ns]', name=u'tpep_pickup_datetime', length=719, freq=None)

In [147]:

#predDf['2016-06-01 01:00:00':'2016-06-30 17:00:00']
june_from = pd.to_datetime('2016-06-01 00:00:00')
june_to = pd.to_datetime('2016-06-30 18:00:00')

In [148]:
print len(predDf[june_from :june_to].index)
print len(juneDf['1075_'+str(shift + 1)].sort_index().values)
print [1,2,3][:-1:]

715
715
[1, 2]


In [178]:
predResults = pd.DataFrame()

for reg in data.region.unique():
    predDf = pd.DataFrame(index=s.index)
    for shift in arange(6):
        p = juneDf[str(reg) + "_" + str(shift + 1)].sort_index()
        t_delta = datetime.timedelta(hours=shift)

        print len(p), len(predDf[june_from + t_delta:june_to + t_delta].index)
        predDf[str(shift + 1)] = np.nan
        predDf[str(shift + 1)][june_from + t_delta:june_to + t_delta] = p.values[:-1:] if shift == 5 else p.values 
        predResults[str(reg)] = predDf.T.fillna(predDf.mean(axis=1)).T.mean(axis=1)
    

715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715
715 714
715 715
715 715
715 715
715 715
715 715


In [186]:
maes = []
for reg in data.region.unique():
    mae = metrics.mean_absolute_error(predResults[str(reg)], 
                                  data[data.region == reg]
                                  ['2016-06-01 00:00:00':'2016-06-30 22:00:00']['count'].sort_index().values)
    maes.append(mae)
    #print mae
print np.mean(maes)

15.686591068689161


In [159]:
pdf =  predDf.T.fillna(predDf.mean(axis=1)).T


In [174]:
print metrics.mean_absolute_error(pdf.mean(axis=1).values, s.values)
print metrics.mean_absolute_error(juneDf['1075_1'].sort_index().values, s[june_from:june_to].values)


10.694575344554574
10.003183974252714


In [184]:
#predResults.to_pickle('TaxiData/w7_predictions.dat')

In [196]:
predResults

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1733,1734,1783,2068,2069,2118,2119,2168,1684
tpep_pickup_datetime,,,,,,,,,,,,,,,,,,,,,
2016-06-01 00:00:00,26.614845,35.374245,23.230392,35.150288,75.534286,151.949295,231.815781,225.750824,313.648773,72.539551,...,5.105243,4.598626,250.814377,101.174118,99.436043,9.837701,174.202240,12.987780,81.185265,0.560961
2016-06-01 01:00:00,12.775550,23.578528,7.535903,23.421752,47.250961,103.438538,119.089874,151.142036,228.201004,53.075863,...,6.364617,2.378320,87.459116,9.182878,30.629736,4.686229,78.962341,11.054438,51.909765,0.424790
2016-06-01 02:00:00,6.297398,10.671741,2.146412,13.896968,24.971421,58.262071,70.029069,135.030706,247.996902,22.535304,...,5.932507,0.174487,5.975630,0.559307,6.372814,1.053412,19.615829,15.732163,2.314129,0.031919
2016-06-01 03:00:00,3.955818,10.895498,1.654717,7.715266,17.481605,35.556719,45.588594,95.758200,202.673275,18.658936,...,5.678134,0.527743,10.911064,4.659839,3.387384,0.642520,28.636694,4.944586,-0.830300,0.736587
2016-06-01 04:00:00,5.928780,10.899874,1.755322,6.907360,23.614899,30.340596,37.881979,75.925938,140.095541,15.450590,...,6.480461,0.294886,3.173128,4.720439,3.776718,0.840997,25.978908,0.903941,4.184681,0.381149
2016-06-01 05:00:00,9.159670,24.052444,13.505313,8.319912,39.211725,48.726455,62.600609,84.042797,83.484500,17.267507,...,5.852757,0.511821,4.402825,1.503752,20.431939,2.965231,75.277874,2.154148,63.350981,0.350202
2016-06-01 06:00:00,20.245141,68.267497,75.867931,43.020090,106.706179,133.781837,141.397523,214.988202,123.999111,47.616330,...,6.011230,2.675648,12.422992,17.878003,61.497531,8.955553,155.725690,10.106240,86.936291,4.836126
2016-06-01 07:00:00,44.006516,138.129912,158.661583,72.436731,152.879430,261.207031,231.601944,368.275503,201.241918,108.317159,...,7.718032,7.572683,98.694417,187.540993,67.480784,6.411519,136.237350,21.637447,77.452526,9.307242
2016-06-01 08:00:00,77.069564,201.919767,217.923205,93.519430,193.411041,322.457977,287.376582,477.114970,290.411982,196.213654,...,5.413342,8.090587,200.574969,217.653386,46.198379,5.927909,98.906638,6.024360,67.734799,12.266878


In [190]:
juneData = data['2016-06-01 00:00:00':'2016-06-30 22:00:00']


In [194]:
juneDataByReg = pd.DataFrame()
for reg in data.region.unique():
    regData = juneData[juneData.region == reg]['count']
    juneDataByReg[str(reg)] = regData

In [197]:
juneDataByReg.to_pickle('TaxiData/w7_target.dat')